# All features extracted by Librosa 


- Zero Crossing Rate, (mean and var)
- Harmonics (mean and var)
- Perceptrual (mean and var)
- Tempo
- Spectral Centroid (mean and var)
- Spectral Rolloff (mean and var)
- Spectral Bandwidth (mean and var)
- Mel-Frequency Cepstral Coefficients (20 different coefficients) (mean and var)
- Chroma (mean and var)
- rms energy (mean and var)
- lenghth of the audio file.

**Note on Librosa** 
- Librosa can open several audio formats beyond just WAV files. It uses backends like PySoundFile and audioread, which support formats such as MP3, FLAC, OGG, and more. Just ensure you have the necessary dependencies (like FFmpeg, libsndfile, etc.) installed for your system.

# Code for extracting features from one audio file 

Librosa Conversion code below ⤵  (for one audio file, as defined by the audio path) 

In [15]:
import os
print(os.getcwd())


/home/tford/code/zmokhtari89/k_means_klang/Notebooks/librosa


In [23]:
#Import
import librosa
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

general_path = '../../raw_data/Data'

#when in the .py we will ned to use os.join__file__ and then set the path 

# Define the path to the audio file
file_path = f'{general_path}/genres_original/jazz/jazz.00055.wav'

#------------------------------------------------------------
#Step 1:  # Step 1: Load and trim the audio file
#------------------------------------------------------------
y, sr = librosa.load(str(file_path)) 
audio_file, _ = librosa.effects.trim(y)

#------------------------------------------------------------
#Step 2: Extract features. When relevant, calculate mean and variance
#------------------------------------------------------------

# Length of the audio file (in samples)
length = audio_file.shape[0]

# Zero Crossing Rate
zero_crossings = librosa.zero_crossings(audio_file, pad=False)
zero_crossing_rate_mean = np.mean(zero_crossings)
zero_crossing_rate_var = np.var(zero_crossings)

# Harmonics & Percussive Components (HPSS)
y_harm, y_perc = librosa.effects.hpss(audio_file)
harmony_mean = np.mean(y_harm)
harmony_var = np.var(y_harm)
perceptr_mean = np.mean(y_perc)
perceptr_var = np.var(y_perc)

#Tempo: 
tempo_value, _ = librosa.beat.beat_track(y=audio_file, sr = sr) # a 2nd variable is created by the function, but we ignore it with the space' _ '.
tempo = tempo_value.item()

# Spectral Centroid
spectral_centroids = librosa.feature.spectral_centroid(y=audio_file, sr=sr)[0]
spectral_centroid_mean = np.mean(spectral_centroids)
spectral_centroid_var = np.var(spectral_centroids)

# Spectral Rolloff
spectral_rolloff = librosa.feature.spectral_rolloff(y=audio_file, sr=sr)[0]
rolloff_mean = np.mean(spectral_rolloff)
rolloff_var = np.var(spectral_rolloff)

# Spectral Bandwidth
bandwidth = librosa.feature.spectral_bandwidth(y=audio_file, sr=sr)
spectral_bandwidth_mean = np.mean(bandwidth)
spectral_bandwidth_var = np.var(bandwidth)

#Chroma Frequencies (short-time fourier transform): 
hop_length = 5000   #Increase or decrease hop_length to change how granular you want your data to be
chromagram = librosa.feature.chroma_stft(y=audio_file, sr=sr, hop_length=hop_length)
chroma_stft_mean = np.mean(chromagram)
chroma_stft_var = np.var(chromagram)

# RMS Energy
rms_values = librosa.feature.rms(y=audio_file)
rms_mean = np.mean(rms_values)
rms_var = np.var(rms_values)

# Mel-Frequency Cepstral Coefficients (MFCCs)
mfccs = librosa.feature.mfcc(y=audio_file, sr=sr)
mfcc_means = np.mean(mfccs, axis=1)  # Array of 20 means
mfcc_vars = np.var(mfccs, axis=1)     # Array of 20 variances


#------------------------------------------------------------
#Step 3: Create a dictionary with all the features
#------------------------------------------------------------

data = {
    'file_name': file_path.split('/')[-1],
    'length_samples': length,
    'zero_crossing_rate_mean': zero_crossing_rate_mean,
    'zero_crossing_rate_var': zero_crossing_rate_var,
    'harmony_mean': harmony_mean,
    'harmony_var': harmony_var,
    'perceptr_mean': perceptr_mean,
    'perceptr_var': perceptr_var,
    'tempo': tempo,
    'spectral_centroid_mean': spectral_centroid_mean,
    'spectral_centroid_var': spectral_centroid_var,
    'spectral_rolloff_mean': rolloff_mean,
    'spectral_rolloff_var': rolloff_var,
    'spectral_bandwidth_mean': spectral_bandwidth_mean,
    'spectral_bandwidth_var': spectral_bandwidth_var,
    'chroma_stft_mean': chroma_stft_mean,
    'chroma_stft_var': chroma_stft_var,
    'rms_mean': rms_mean,
    'rms_var': rms_var
}

# Add MFCC features as separate columns
for i in range(len(mfcc_means)):
    data[f'mfcc_mean_{i+1}'] = mfcc_means[i]
    data[f'mfcc_var_{i+1}'] = mfcc_vars[i]

# Step 4: Create a pandas DataFrame
df = pd.DataFrame([data])

#Step 5: Apply Standard Scaling to all numeric columns 
scaler = MinMaxScaler()
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns]= scaler.fit_transform(df[numeric_columns])

df.head()



,file_name,length_samples,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,perceptr_mean,perceptr_var,tempo,spectral_centroid_mean,...,mfcc_mean_16,mfcc_var_16,mfcc_mean_17,mfcc_var_17,mfcc_mean_18,mfcc_var_18,mfcc_mean_19,mfcc_var_19,mfcc_mean_20,mfcc_var_20
0,jazz.00055.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Explanation:
- Audio Loading and Trimming: The file is loaded and trimmed using librosa.load and librosa.effects.trim.
- Feature Extraction: Various features are extracted (mean and variance where applicable) using librosa functions.
- MFCCs: The MFCC coefficients are processed along axis 1 to yield a mean and variance for each of the 20 coefficients.
- Dictionary Creation: All features are stored in a dictionary. The file name is extracted from the file path.
- DataFrame Creation: A DataFrame is created from the dictionary, resulting in a single-row DataFrame that encapsulates all the extracted features.

# Code for extracting features for multiple audio files in a folder (and subfolders)
- for various types of audio formats (see librosa note above)

**Optimized with parallell processing**  
takes appox 15 mins

In [27]:
%%time

import os
import librosa
import numpy as np
import pandas as pd
from pathlib import Path
from joblib import Parallel, delayed
from sklearn.preprocessing import MinMaxScaler

#defining a function to extract features 
def extract_features(file_path):
    # Load and trim the audio file
    y, sr = librosa.load(file_path)
    audio_file, _ = librosa.effects.trim(y)
    
    # Length (in samples)
    length = audio_file.shape[0]
    
    # Zero Crossing Rate
    zero_crossings = librosa.zero_crossings(audio_file, pad=False)
    zero_crossings_rate_mean = np.mean(zero_crossings)
    zero_crossings_rate_var = np.var(zero_crossings)
    
    # Harmonics & Percussive Components (HPSS)
    y_harm, y_perc = librosa.effects.hpss(audio_file)
    harmony_mean = np.mean(y_harm)
    harmony_var = np.var(y_harm)
    perceptr_mean = np.mean(y_perc)
    perceptr_var = np.var(y_perc)
    
    # Tempo
    tempo_value, _ = librosa.beat.beat_track(y=audio_file, sr=sr)
    tempo = tempo_value.item()
    
    # Spectral Centroid
    spectral_centroids = librosa.feature.spectral_centroid(y=audio_file, sr=sr)[0]
    spectral_centroid_mean = np.mean(spectral_centroids)
    spectral_centroid_var = np.var(spectral_centroids)
    
    # Spectral Rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio_file, sr=sr)[0]
    rolloff_mean = np.mean(spectral_rolloff)
    rolloff_var = np.var(spectral_rolloff)
    
    # Spectral Bandwidth
    bandwidth = librosa.feature.spectral_bandwidth(y=audio_file, sr=sr)
    spectral_bandwidth_mean = np.mean(bandwidth)
    spectral_bandwidth_var = np.var(bandwidth)
    
    # Chroma Frequencies
    hop_length = 5000  # Adjust for granularity
    chromagram = librosa.feature.chroma_stft(y=audio_file, sr=sr, hop_length=hop_length)
    chroma_mean = np.mean(chromagram)
    chroma_var = np.var(chromagram)
    
    # RMS Energy
    rms_values = librosa.feature.rms(y=audio_file)
    rms_mean = np.mean(rms_values)
    rms_var = np.var(rms_values)
    
    # MFCCs (20 coefficients)
    mfccs = librosa.feature.mfcc(y=audio_file, sr=sr)
    mfcc_means = np.mean(mfccs, axis=1)
    mfcc_vars = np.var(mfccs, axis=1)
    
    # Build feature dictionary
    features = {
        'file_name': os.path.basename(file_path),
        'length_samples': length,
        'zero_crossings_rate_mean': zero_crossings_rate_mean,
        'zero_crossings_rate_var': zero_crossings_rate_var,
        'harmony_mean': harmony_mean,
        'harmony_var': harmony_var,
        'perceptr_mean': perceptr_mean,
        'perceptr_var': perceptr_var,
        'tempo': tempo,
        'spectral_centroid_mean': spectral_centroid_mean,
        'spectral_centroid_var': spectral_centroid_var,
        'rolloff_mean': rolloff_mean,
        'rolloff_var': rolloff_var,
        'spectral_bandwidth_mean': spectral_bandwidth_mean,
        'spectral_bandwidth_var': spectral_bandwidth_var,
        'chroma_mean': chroma_mean,
        'chroma_var': chroma_var,
        'rms_mean': rms_mean,
        'rms_var': rms_var
    }
    
    # Add MFCC features (20 coefficients)
    for i in range(len(mfcc_means)):
        features[f'mfcc_mean_{i+1}'] = mfcc_means[i]
        features[f'mfcc_var_{i+1}'] = mfcc_vars[i]
    
    return features

# Define main folder path containing subfolders with audio files
main_folder_path = '../../raw_data/Data/genres_original'

# Collect file paths from all subfolders using os.walk()
file_paths = []
for root, dirs, files in os.walk(main_folder_path):
    for filename in files:
        if filename.lower().endswith(('.wav', '.mp3', '.flac')):
            file_paths.append(os.path.join(root, filename))

# Use joblib to process files in parallel
data_list = Parallel(n_jobs=-1)(delayed(extract_features)(fp) for fp in file_paths)

# Create a DataFrame from the list of feature dictionaries
df= pd.DataFrame(data_list)





CPU times: user 14.6 s, sys: 3.22 s, total: 17.8 s
Wall time: 18min 37s


### Scaling: 

In [ ]:
#Apply Scaling to all numeric columns 
scaler = MinMaxScaler()
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns]= scaler.fit_transform(df[numeric_columns])

In [35]:
#create a csv file 
# df.to_csv('features_30_sec_taitest.csv', index=False) 